In [13]:


import sparknlp
 
spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)



Spark NLP version 3.1.0
Apache Spark version: 3.0.2


In [14]:


from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng_custom.train')
testing_data= CoNLL().readDataset(spark, './eng_custom.testa')



In [15]:
import pyspark.sql.functions as F

training_data.select(F.explode(F.arrays_zip('token.result', 'pos.result',  'label.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("pos"),
        F.expr("cols['2']").alias("ner_label")).show(truncate=50)


+--------+---+---------+
|   token|pos|ner_label|
+--------+---+---------+
|    make|  0|      Act|
|     can|  0|      Act|
|      do|  0|      Act|
|    read|  0|      Act|
|   write|  0|      Act|
|    talk|  0|      Act|
|    want|  0|      Act|
|    cook|  0|      Act|
|    brew|  0|      Act|
|   clean|  0|      Act|
|   brush|  0|      Act|
|    care|  0|      Act|
|    walk|  0|      Act|
|      go|  0|      Act|
|navigate|  0|      Act|
|activity|  0|      Act|
|   write|  0|      Act|
|    move|  0|      Act|
|    fall|  0|      Act|
|      up|  0|      Act|
+--------+---+---------+
only showing top 20 rows



In [16]:
!mkdir ner_logs

mkdir: cannot create directory ‘ner_logs’: File exists


In [17]:


# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)

embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(100)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs
 #    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
 #    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch 
    
ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

ner_pipeline = Pipeline(stages=[
      embeddings,
      nerTagger,
      ner_converter
 ])



glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [18]:
%%time
ner_model = ner_pipeline.fit(training_data)
ner_model.write().overwrite().save('outputs/ner_wiki_glove100d_en')

2022-10-13 19:36:20.195650: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

Training started - total epochs: 100 - lr: 0.003 - batch size: 32 - labels: 7 - chars: 25 - training examples: 136
Epoch 1/100 started, lr: 0.003, dataset size: 136
Epoch 1/100 - 0.61s - loss: 15.865172 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.16s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 3	 0	 10	 1.0	 0.23076923	 0.375
Act	 12	 45	 2	 0.21052632	 0.85714287	 0.33802816
0	 9	 1	 33	 0.9	 0.21428572	 0.34615386
Obj	 8	 4	 4	 0.6666667	 0.6666667	 0.6666667
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 32 fp: 50 fn: 50 labels: 5
Macro-average	 prec: 0.55543864, rec: 0.3937729, f1: 0.46083865
Micro-average	 prec: 0.3902439, rec: 0.3902439, f1: 0.39024386
Epoch 2/100 started, lr: 0.0029850747, dataset size: 136
Epoch 2/100 - 0.08s - loss: 17.465513 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 0	 0	 13	 0.0	 0.0	 0.0
Act	 6	 6	 8	 0.5	 0.4285

Epoch 17/100 - 0.07s - loss: 0.68950915 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 12	 0	 1	 1.0	 0.9230769	 0.96000004
Act	 12	 3	 2	 0.8	 0.85714287	 0.82758623
0	 41	 3	 1	 0.9318182	 0.97619045	 0.9534884
Obj	 9	 2	 3	 0.8181818	 0.75	 0.78260875
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 74 fp: 8 fn: 8 labels: 5
Macro-average	 prec: 0.71, rec: 0.701282, f1: 0.70561403
Micro-average	 prec: 0.902439, rec: 0.902439, f1: 0.902439
Epoch 18/100 started, lr: 0.0027649768, dataset size: 136
Epoch 18/100 - 0.07s - loss: 0.4422918 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 12	 0	 1	 1.0	 0.9230769	 0.96000004
Act	 12	 3	 2	 0.8	 0.85714287	 0.82758623
0	 41	 3	 1	 0.9318182	 0.97619045	 0.9534884
Obj	 9	 2	 3	 0.8181818	 0.75	 0.78260875
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 74 fp: 8 fn: 8 labels: 5
M

Epoch 32/100 - 0.07s - loss: 0.20076993 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 12	 0	 1	 1.0	 0.9230769	 0.96000004
Act	 12	 3	 2	 0.8	 0.85714287	 0.82758623
0	 41	 3	 1	 0.9318182	 0.97619045	 0.9534884
Obj	 9	 2	 3	 0.8181818	 0.75	 0.78260875
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 74 fp: 8 fn: 8 labels: 5
Macro-average	 prec: 0.71, rec: 0.701282, f1: 0.70561403
Micro-average	 prec: 0.902439, rec: 0.902439, f1: 0.902439
Epoch 33/100 started, lr: 0.002586207, dataset size: 136
Epoch 33/100 - 0.07s - loss: 0.087027974 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 12	 0	 1	 1.0	 0.9230769	 0.96000004
Act	 12	 2	 2	 0.85714287	 0.85714287	 0.85714287
0	 41	 3	 1	 0.9318182	 0.97619045	 0.9534884
Obj	 10	 2	 2	 0.8333333	 0.8333333	 0.8333333
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 75 fp: 7 fn: 

Epoch 47/100 - 0.07s - loss: 0.02653329 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 12	 0	 1	 1.0	 0.9230769	 0.96000004
Act	 12	 4	 2	 0.75	 0.85714287	 0.79999995
0	 41	 1	 1	 0.97619045	 0.97619045	 0.97619045
Obj	 10	 2	 2	 0.8333333	 0.8333333	 0.8333333
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 75 fp: 7 fn: 7 labels: 5
Macro-average	 prec: 0.71190476, rec: 0.7179487, f1: 0.71491396
Micro-average	 prec: 0.91463417, rec: 0.91463417, f1: 0.91463417
Epoch 48/100 started, lr: 0.0024291498, dataset size: 136
Epoch 48/100 - 0.07s - loss: 0.025760725 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 12	 0	 1	 1.0	 0.9230769	 0.96000004
Act	 12	 4	 2	 0.75	 0.85714287	 0.79999995
0	 41	 1	 1	 0.97619045	 0.97619045	 0.97619045
Obj	 10	 2	 2	 0.8333333	 0.8333333	 0.8333333
Pos	 0	 0	 1	 0.0	 0.0	 0.0

Epoch 62/100 - 0.07s - loss: 0.016180452 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 13	 4	 1	 0.7647059	 0.9285714	 0.83870965
0	 41	 4	 1	 0.9111111	 0.97619045	 0.9425287
Obj	 7	 0	 5	 1.0	 0.5833333	 0.7368421
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 74 fp: 8 fn: 8 labels: 5
Macro-average	 prec: 0.7351634, rec: 0.697619, f1: 0.71589935
Micro-average	 prec: 0.902439, rec: 0.902439, f1: 0.902439
Epoch 63/100 started, lr: 0.0022900763, dataset size: 136
Epoch 63/100 - 0.07s - loss: 0.110729046 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 12	 3	 2	 0.8	 0.85714287	 0.82758623
0	 41	 3	 1	 0.9318182	 0.97619045	 0.9534884
Obj	 9	 1	 3	 0.9	 0.75	 0.81818175
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 75 fp: 7 fn: 7 labels: 5
Macro-average	 prec: 0

Epoch 77/100 - 0.08s - loss: 0.0062551745 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 12	 3	 2	 0.8	 0.85714287	 0.82758623
0	 41	 3	 1	 0.9318182	 0.97619045	 0.9534884
Obj	 9	 1	 3	 0.9	 0.75	 0.81818175
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 75 fp: 7 fn: 7 labels: 5
Macro-average	 prec: 0.72636366, rec: 0.7166667, f1: 0.7214826
Micro-average	 prec: 0.91463417, rec: 0.91463417, f1: 0.91463417
Epoch 78/100 started, lr: 0.002166065, dataset size: 136
Epoch 78/100 - 0.07s - loss: 0.0070385784 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 12	 3	 2	 0.8	 0.85714287	 0.82758623
0	 41	 3	 1	 0.9318182	 0.97619045	 0.9534884
Obj	 9	 1	 3	 0.9	 0.75	 0.81818175
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 75 fp: 7 fn: 7 labels: 5
Macro-average	 prec: 0.

Epoch 92/100 - 0.07s - loss: 0.00437362 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 12	 3	 2	 0.8	 0.85714287	 0.82758623
0	 41	 3	 1	 0.9318182	 0.97619045	 0.9534884
Obj	 9	 1	 3	 0.9	 0.75	 0.81818175
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 75 fp: 7 fn: 7 labels: 5
Macro-average	 prec: 0.72636366, rec: 0.7166667, f1: 0.7214826
Micro-average	 prec: 0.91463417, rec: 0.91463417, f1: 0.91463417
Epoch 93/100 started, lr: 0.0020547945, dataset size: 136
Epoch 93/100 - 0.07s - loss: 0.0064352155 - batches: 5
Quality on validation dataset (20.0%), validation examples = 27
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Loc	 13	 0	 0	 1.0	 1.0	 1.0
Act	 12	 4	 2	 0.75	 0.85714287	 0.79999995
0	 41	 2	 1	 0.95348835	 0.97619045	 0.9647058
Obj	 9	 1	 3	 0.9	 0.75	 0.81818175
Pos	 0	 0	 1	 0.0	 0.0	 0.0
tp: 75 fp: 7 fn: 7 labels: 5
Macro-average	 prec: 0

2022-10-13 19:36:29.815846: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

CPU times: user 107 ms, sys: 38.5 ms, total: 146 ms
Wall time: 12.8 s


In [19]:
import pyspark.sql.functions as F
predictions = ner_model.transform(testing_data) 
predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
           .select(F.expr("cols['0']").alias("token"),
                   F.expr("cols['1']").alias("ground_truth"),
                   F.expr("cols['2']").alias("prediction")).show(truncate=False)

+--------+------------+----------+
|token   |ground_truth|prediction|
+--------+------------+----------+
|I       |0           |0         |
|want    |Act         |Act       |
|to      |0           |0         |
|go      |Act         |Act       |
|to      |0           |0         |
|the     |0           |0         |
|location|Loc         |Loc       |
|I       |0           |0         |
|want    |Act         |Act       |
|to      |0           |0         |
|go      |Act         |Act       |
|to      |0           |0         |
|the     |0           |0         |
|kitchen |Loc         |Loc       |
|I       |0           |0         |
|want    |Act         |Act       |
|to      |0           |0         |
|go      |Act         |Act       |
|to      |0           |0         |
|the     |0           |0         |
+--------+------------+----------+
only showing top 20 rows



In [20]:
from sklearn.metrics import classification_report
 
preds_df = predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
                      .select(F.expr("cols['0']").alias("token"),
                              F.expr("cols['1']").alias("ground_truth"),
                              F.expr("cols['2']").alias("prediction")).toPandas()
 
print(classification_report(preds_df['ground_truth'], preds_df['prediction']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       138
         Act       1.00      1.00      1.00        42
         Loc       1.00      1.00      1.00        36

    accuracy                           1.00       216
   macro avg       1.00      1.00      1.00       216
weighted avg       1.00      1.00      1.00       216



In [21]:
#import shutil

#shutil.make_archive("/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en", 'zip', "/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en")

In [22]:
text_list = [
    """Navagate to kitchen"""
]


In [23]:
from sparknlp.base import LightPipeline

light = LightPipeline(ner_model)
light.annotate("Navigate to kitchen")

{'embeddings': [], 'ner': [], 'ner_chunk': []}

In [24]:




from sparknlp_display import NerVisualizer



result = predictions

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)




